In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [3]:
df = pd.read_csv('../data/2604-prots_pepts.csv', index_col=0)
df.head()

,visit_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,on_med,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVMLPVADQDQCIR,YVMLPVADQDQCIR.1,YVNKEIQNAVNGVK,YWGVASFLQK,YYCFQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,0,10.0,6.0,15.0,1.0,0,11254.3,732430.0,39585.8,...,201158.0,16492.3,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55_3,3,10.0,7.0,25.0,1.0,0,11254.3,732430.0,39585.8,...,201158.0,16492.3,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
2,55_6,6,8.0,10.0,34.0,1.0,0,13163.6,630465.0,35220.8,...,171079.0,13198.8,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
3,55_9,9,8.0,9.0,30.0,0.0,1,13163.6,630465.0,35220.8,...,171079.0,13198.8,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
4,55_12,12,10.0,10.0,41.0,0.0,1,15257.6,815083.0,41650.9,...,231772.0,17873.8,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1


In [4]:
sample = df.sample(200)
sample.head()

,visit_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,on_med,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVMLPVADQDQCIR,YVMLPVADQDQCIR.1,YVNKEIQNAVNGVK,YWGVASFLQK,YYCFQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
1736,42003_54,54,31.0,39.0,15.0,14.0,1,1.00,344975.0,13277.5,...,171701.0,2660.31,4090670.0,59162.80,181286.0,58989.3,93092.2,494082.0,29286.4,16709.9
691,17154_18,18,12.0,4.0,25.0,11.0,1,11270.00,542494.0,25593.5,...,233298.0,1.00,1514180.0,1.00,549497.0,105343.0,157114.0,507382.0,32278.9,1.0
578,14811_48,48,12.0,14.0,21.0,4.0,1,9508.50,248410.0,14013.3,...,305824.0,1.00,1980840.0,2164.54,895891.0,70361.2,86321.7,302315.0,1.0,23671.4
852,20460_24,24,20.0,13.0,23.0,2.0,0,7977.84,389303.0,31827.9,...,216094.0,6814.94,5709810.0,55956.40,971236.0,55903.3,104364.0,493181.0,49443.1,29752.5
1219,27893_0,0,6.0,14.0,26.0,0.0,1,1.00,1.0,1.0,...,1.0,1.00,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
from sklearn.model_selection import train_test_split

def smape(estimator, X_test, y_test):
    return 100/len(y_test) * np.sum(2 * np.abs(estimator.predict(X_test) - y_test) / (np.abs(y_test) + np.abs(estimator.predict(X_test)))) + 1

xgregr = xgb.XGBRegressor(eval_metric=smape)

X = df.drop(['visit_id', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4'], axis=1)
y1 = df[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']]

X_train, X_val, y_train, y_val = train_test_split(X, y1, test_size=0.2)

In [12]:
xgregr_params = {
    'n_estimators': [100, 500],
    'max_depth': [1, 5, 10],
    'gamma': [0, 0.1, 0.5],
    'learning_rate': [0.001, .01, 0.1],
}

from sklearn.model_selection import GridSearchCV

xgregr = xgb.XGBRegressor(eval_metric=smape)
grid = GridSearchCV(xgregr, xgregr_params, scoring=smape, cv=5, n_jobs=2, verbose=3)
grid.fit(X_train, y_train['updrs_1'])

grid.best_params_

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 2/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=100;, score=129.683 total time=   2.6s
[CV 1/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=100;, score=130.866 total time=   2.7s
[CV 3/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=100;, score=128.683 total time=   3.0s
[CV 4/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=100;, score=128.541 total time=   3.1s
[CV 5/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=100;, score=126.034 total time=   2.7s
[CV 1/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=500;, score=84.007 total time=  12.9s
[CV 2/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=500;, score=85.690 total time=  12.7s
[CV 3/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=500;, score=87.024 total time=  12.8s
[CV 4/5] END gamma=0, learning_rate=0.001, max_depth=1, n_estimators=500;, sc

{'gamma': 0, 'learning_rate': 0.001, 'max_depth': 10, 'n_estimators': 100}